In [1]:
import numpy as np
import os
import pandas as pd
import json

import matminer
from matminer.featurizers.site import CrystalNNFingerprint
from pymatgen.core.structure import Structure
from pymatgen.io.cif import CifParser

from scipy.spatial import distance
import heapq
import glob
import csv
import random

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [34]:
dataset_name = 'perovskites'

In [35]:
data_dict = {}
with open(f'data/{dataset_name}/targets.csv', newline='') as csvfile:
    csvreader = csv.reader(csvfile)

    # Iterate over each row in the CSV file
    for row in csvreader:
        if len(row) >= 2:  # Ensure there are at least two columns
            key = row[0]   # First column value
            value = row[1] # Second column value
            data_dict[key] = float(value)

print(data_dict)

{'0': 2.16, '1': 1.52, '2': 1.48, '3': 1.24, '4': 0.62, '5': 1.48, '6': 1.56, '7': 1.2, '8': 1.62, '9': 3.16, '10': 1.4, '11': 2.74, '12': 0.78, '13': 1.24, '14': 1.76, '15': 1.5, '16': 2.06, '17': 1.06, '18': 1.7, '19': 0.46, '20': 1.08, '21': 1.28, '22': 1.76, '23': 1.68, '24': 0.82, '25': 1.16, '26': 1.18, '27': 1.24, '28': 0.32, '29': 0.6, '30': 2.3, '31': 0.66, '32': 2.0, '33': 0.62, '34': 0.98, '35': 1.3, '36': 0.58, '37': 1.18, '38': 1.34, '39': 0.76, '40': 1.16, '41': 1.28, '42': 1.4, '43': 2.96, '44': 0.72, '45': 0.6, '46': 0.68, '47': 0.7, '48': 2.84, '49': 1.18, '50': 0.56, '51': 0.92, '52': 0.98, '53': 3.2, '54': 1.48, '55': 1.36, '56': 1.78, '57': 0.84, '58': 0.36, '59': 0.64, '60': 1.44, '61': 0.0, '62': 1.24, '63': 1.36, '64': 1.9, '65': 1.54, '66': 0.86, '67': 1.48, '68': 1.1, '69': 3.34, '70': 1.08, '71': 3.74, '72': 0.82, '73': 2.3, '74': 3.2, '75': 1.9, '76': 2.98, '77': 1.38, '78': 0.42, '79': 1.0, '80': 1.06, '81': 1.6, '82': 2.04, '83': 0.96, '84': 2.22, '85': 1.9

In [36]:
df = []
count = 0
cif_files = glob.glob(f'data/{dataset_name}/*.cif')
total_files = len(cif_files)
print(total_files)

for i in range(total_files):
    count += 1
    name = f'data/{dataset_name}/{i}.cif'
    parser = CifParser(name)
    struct = parser.get_structures()[0]
    # print(structure)
    # print(struct.composition.reduced_formula)
    
    try:
        fp = CrystalNNFingerprint.from_preset("ops").featurize(struct, 0)
        # print(fp)
        df.append(fp)
    except:
        print(f'no fingerpring found for material {i}')
        continue
    
    if count % 100 == 0:
        print(count)
#         break

df = pd.DataFrame(df)
print(df.shape)

18928


/home/omee/anaconda3/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:4159: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/home/omee/anaconda3/lib/python3.8/site-packages/pymatgen/analysis/local_env.py:3953: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600


/home/omee/anaconda3/lib/python3.8/site-packages/pymatgen/io/cif.py:1150: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18500
18600
18700
18800
18900
(18928, 61)


In [37]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0
1,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0
2,0.000000,0.000000,0.347505,1.485240e-05,3.741148e-03,0.079888,0.312069,0.072892,0.000000,0.000000,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0
3,0.061544,0.061544,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.118443,0.000062,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0
4,0.000000,0.000000,0.014941,1.765264e-14,1.282971e-09,0.000002,0.002273,0.014876,0.010901,0.000203,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0


In [38]:
dist = distance.cdist(df.values, df.values, "euclidean")
print(dist)

[[0.         2.08990616 1.78151482 ... 0.1394488  2.12482207 2.00640798]
 [2.08990616 0.         0.93501232 ... 1.95916206 2.07425221 0.42768585]
 [1.78151482 0.93501232 0.         ... 1.65291778 1.76312493 0.74904513]
 ...
 [0.1394488  1.95916206 1.65291778 ... 0.         2.04826244 1.87884891]
 [2.12482207 2.07425221 1.76312493 ... 2.04826244 0.         1.99009735]
 [2.00640798 0.42768585 0.74904513 ... 1.87884891 1.99009735 0.        ]]


In [39]:
new_dist = [] 
for i in range(len(dist)):
    ## find 3 neighbors
    tmp = heapq.nsmallest(3, dist[i])
    new_dist.append(sum(tmp)/3.0)

max_number = heapq.nlargest(500, new_dist) 
print(max_number)

max_idx = []
for t in max_number:
    max_idx.append(new_dist.index(t))
print(max_idx)

[0.33088196523703617, 0.2769024819820796, 0.2624419143418983, 0.2467219797945639, 0.2284764010069813, 0.2252556938840342, 0.2198449911168698, 0.20562938844626655, 0.19087449751776187, 0.1852704318424685, 0.18509949382422153, 0.18413864550829126, 0.1832880355692579, 0.18202999606089545, 0.18097264722526196, 0.17996184841980092, 0.17667329464972248, 0.17267231129324365, 0.1717734067678093, 0.1705772925090959, 0.168398840749958, 0.16737315452824397, 0.1649912297373292, 0.16391702152433077, 0.15800672422807016, 0.1579979013465339, 0.15703086577276945, 0.15509669155535685, 0.15391207145107708, 0.15381210530221642, 0.15284542201623363, 0.15246187876438666, 0.15181343838613073, 0.15052697432558435, 0.14931093495114142, 0.147331986525399, 0.1463109913878737, 0.14616604275979092, 0.1456272665295321, 0.14515726285487826, 0.14515227964036148, 0.14420432990814, 0.14401428014923542, 0.1435275986779914, 0.1426813022993527, 0.14138569363018094, 0.13997943991527576, 0.1391169562178097, 0.1391051533721

In [40]:
# ood = []
# rest = []

# cc = 0
# for i in range(len(df)):
#     if i in max_idx:
#         ood.append(list(df.iloc[i]))
#     else:
#         rest.append(list(df.iloc[i]))

# ood = pd.DataFrame(ood, columns=df.columns)
# rest = pd.DataFrame(rest, columns=df.columns)

# if len(rest) > 4500:
#     idx = random.sample(range(len(rest)), 4200) 
#     rest = rest.loc[idx]

# print(ood, rest)
# print(len(ood), len(rest))

In [41]:
train_fold_path = f'folds/id/{dataset_name}/train.json'
val_fold_path = f'folds/id/{dataset_name}/val.json'
test_fold_path = f'folds/id/{dataset_name}/test.json'

In [42]:
# train_folds = {}
# val_folds = {}

# test_folds = {}
# folds = 5

# for fold in range(folds):
#     test = []
#     rest = []

#     for i in range(len(df)):
#         if i in max_idx:
#             test.append(i)
#         else:
#             rest.append(i)

#     # if len(rest) > 4000:
#     idx = random.sample(range(len(rest)), 4000) 
#     train = [rest[i] for i in idx]
#     val = list(set(rest) - set(train))
#     if len(val) > 500:
#         idx = random.sample(range(len(val)), 500)
#         val = [val[i] for i in idx]

# #     print(test, train, val)
#     print(len(test), len(train), len(val))
    
#     train_folds[str(fold)] = train
#     val_folds[str(fold)] = val
#     test_folds[str(fold)] = test

In [43]:
train_folds = {}
val_folds = {}
test_folds = {}

folds = 5

for fold in range(folds):
    ood_test = []
    rest = []

    for i in range(len(df)):
        if i in max_idx:
            ood_test.append(i)
        else:
            rest.append(i)

    idx = random.sample(range(len(rest)), 4000) 
    train = [rest[i] for i in idx]
    
    val = list(set(rest) - set(train))
    idx = random.sample(range(len(val)), 500)
    val = [val[i] for i in idx]
    
    test = list(set(rest) - set(train))
    test = list(set(test) - set(val))
    
    if len(test) > 500:
        idx = random.sample(range(len(test)), 500)
        test = [test[i] for i in idx]

#     print(test, train, val)
    print(len(test), len(train), len(val))
    
    train_folds[str(fold)] = train
    val_folds[str(fold)] = val
    test_folds[str(fold)] = test

500 4000 500
500 4000 500
500 4000 500
500 4000 500
500 4000 500


In [44]:
# def check_overlap(list1, list2):
#     # Convert lists to sets
#     set1 = set(list1)
#     set2 = set(list2)
    
#     # Find the intersection of two sets
#     overlap = set1.intersection(set2)
    
#     # Check if there is any overlap
#     return len(overlap) > 0

In [45]:
# check_overlap(train, val), check_overlap(test, val), check_overlap(train, test)

In [46]:
with open(train_fold_path, 'w') as file:
    json.dump(train_folds, file)

with open(val_fold_path, 'w') as file:
    json.dump(val_folds, file)

with open(test_fold_path, 'w') as file:
    json.dump(test_folds, file)

In [32]:
maes = []
mses = []
rmses = []
r2s = []
for i in range(5):
    print(f'run {i}\n --------------------------')
    df = pd.read_csv(f'results/pe_ime_{i}.csv')
    y_true = np.array(df['target'])
    y_pred = np.array(df['prediction'])
    
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared = False)
    mse = mean_squared_error(y_true, y_pred, squared = True)
    r2 = r2_score(y_true, y_pred)
    print(f'MAE: {mae:.4f}')
    print(f'MSE: {mse:.4f}')
    print(f'R2: {r2:.4f}\n')
    maes.append(mae)
    mses.append(mse)
    rmses.append(rmse)
    r2s.append(r2)

# Calculate mean and standard deviation
# mean_mae = np.mean(maes)
# std_mae = np.std(maes)

# mean_mse = np.mean(mses)
# std_mse = np.std(mses)

# mean_r2 = np.mean(r2s)
# std_r2 = np.std(r2s)


# Format as mean ± standard deviation
# formatted_mae = f"${mean_mae:.4f} \pm {std_mae:.4f}$"
# formatted_mse = f"${mean_mse:.4f} \pm {std_mse:.4f}$"
# formatted_r2 = f"${mean_r2:.4f} \pm {std_r2:.4f}$"

# print(f'MAE: {formatted_mae}')
# print(f'MSE: {formatted_mse}')
# print(f'R2 Score: {formatted_r2}')

print(f'MAE: {sum(maes)/5:.4f}')
print(f'MSE: {sum(mses)/5:.4f}')
print(f'RMSE: {sum(rmses)/5:.4f}')
print(f'R2 Score: {sum(r2s)/5:.4f}')

run 0
 --------------------------
MAE: 0.0628
MSE: 0.0081
R2: 0.9854

run 1
 --------------------------
MAE: 0.0614
MSE: 0.0082
R2: 0.9844

run 2
 --------------------------
MAE: 0.0648
MSE: 0.0096
R2: 0.9792

run 3
 --------------------------
MAE: 0.0604
MSE: 0.0091
R2: 0.9819

run 4
 --------------------------
MAE: 0.0579
MSE: 0.0067
R2: 0.9879

MAE: 0.0615
MSE: 0.0083
RMSE: 0.0911
R2 Score: 0.9838
